In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import yfinance as yf
from sklearn.model_selection import cross_val_score

# Function to create dataset for time series forecasting
ticker = 'RELIANCE.NS'  # Example: Reliance Industries Limited
data = yf.download(ticker, start='2010-01-01', end='2024-11-01')
print(data.tail())  # Print the first few rows of the data
print(data.shape)

# Preprocess the data
data = data[['Close']]
data = data.astype('float32')

# Add technical indicators
def add_technical_indicators(data):
    data['SMA'] = data['Close'].rolling(window=14).mean()  # Simple Moving Average
    data['Volatility'] = data['Close'].rolling(window=14).std()  # Standard deviation
    data = data.dropna()  # Drop NA values after rolling calculations
    return data

data = add_technical_indicators(data)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

# Create a dataset
def create_dataset(dataset, time_step=1):
    X, Y = [], []
    for i in range(len(dataset) - time_step):
        X.append(dataset[i:(i + time_step)])
        Y.append(dataset[i + time_step, 0])  # Target variable
    return np.array(X), np.array(Y)

time_step = 60  # Look back 60 days
X, y = create_dataset(data_scaled, time_step)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Train the Stacking Regressor
estimators = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('xgb', XGBRegressor(objective='reg:squarederror')),
    ('lr', LinearRegression())
]

stacking_model = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
stacking_model.fit(X_train.reshape(X_train.shape[0], -1), y_train)

# Cross-validation
cross_val_scores = cross_val_score(stacking_model, X_train.reshape(X_train.shape[0], -1), y_train, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-validated RMSE: {np.sqrt(-cross_val_scores.mean()):.2f}')

# Get predictions
predictions = stacking_model.predict(X_test.reshape(X_test.shape[0], -1))
predictions = scaler.inverse_transform(predictions.reshape(-1, 1))  # Inverse the scaling

# Calculate metrics
rmse = np.sqrt(mean_squared_error(y_test, predictions))
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

# Print the metrics
print(f'RMSE: {rmse:.2f}')
print(f'MAE: {mae:.2f}')
print(f'R² Score: {r2:.2f}')

# Plotting the results
plt.figure(figsize=(14, 5))
plt.plot(data[len(data) - len(predictions):], label='Actual Prices', color='blue')
plt.plot(predictions, label='Predicted Prices', color='red')
plt.title(f'Stock Price Prediction for {ticker} using Stacking Regressor (RMSE: {rmse:.2f})')
plt.xlabel('Days')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

[*********************100%***********************]  1 of 1 completed


Price                        Adj Close        Close         High          Low  \
Ticker                     RELIANCE.NS  RELIANCE.NS  RELIANCE.NS  RELIANCE.NS   
Date                                                                            
2024-10-25 00:00:00+00:00  1327.849976  1327.849976  1344.349976  1322.000000   
2024-10-28 00:00:00+00:00  1334.349976  1334.349976  1353.000000  1322.099976   
2024-10-29 00:00:00+00:00  1340.000000  1340.000000  1343.199951  1320.300049   
2024-10-30 00:00:00+00:00  1343.900024  1343.900024  1350.000000  1325.349976   
2024-10-31 00:00:00+00:00  1332.050049  1332.050049  1343.000000  1326.150024   

Price                             Open      Volume  
Ticker                     RELIANCE.NS RELIANCE.NS  
Date                                                
2024-10-25 00:00:00+00:00  1343.500000    18597496  
2024-10-28 00:00:00+00:00  1337.000000    10824350  
2024-10-29 00:00:00+00:00  1328.099976    12008361  
2024-10-30 00:00:00+00:00  1335.0

ValueError: non-broadcastable output operand with shape (718,1) doesn't match the broadcast shape (718,3)